# Bader charge analysis

For the source code, see [bader](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.bader.bader_workgraph).




## Example: H2O

### Prepare the inputs and visualize the WorkGraph


In [1]:
from ase.build import molecule
from aiida import load_profile

from workgraph_collections.ase.espresso.bader import BaderWorkgraph
from aiida_pythonjob.config import config
# allow pickle
config["allow_pickle"] = True

load_profile()

atoms = molecule('H2O', vacuum=3.0)
atoms.pbc = True


metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"H": "H.pbe-kjpaw_psl.1.0.0.UPF", 
                    "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo/kjpaw"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
scf_input_data = {
    "control": {"calculation": "scf",
                },
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold"
                },
}

inputs = {
    "scf": {
        "input_data": scf_input_data,
        "kpts": [1, 1, 1],
        "metadata": metadata
    },
    "pp_valence": {
        "metadata": metadata
    },
    "pp_all": {
        "metadata": metadata
    },
    "bader": {
        "metadata": metadata
    }
}
#------------------------- Set the inputs -------------------------
wg = BaderWorkgraph.build_graph(atoms=atoms,
                    pw_command="mpirun -np 1 pw.x",
                    pp_command="mpirun -np 1 pp.x",
                    bader_command="bader",
                    computer="localhost",
                    pseudopotentials=pseudopotentials,
                    pseudo_dir=pseudo_dir,
                    inputs=inputs)
wg.to_html()

### Run the WorkGraph



In [2]:
#------------------------- Submit the calculation -------------------
wg.run()


09/04/2025 07:17:28 AM <564861> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179998|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179998|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
09/04/2025 07:17:29 AM <564861> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179998|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 180002
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179998|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 180002
09/04/2025 07:17:46 AM <564861> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [179998|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[179998|WorkGraphEngine|update_task_state]: Task: pw_calculator, type: PYTHONJOB, finishe

In [6]:
#------------------------- Print the output -------------------------
charges = wg.outputs.result.value.value
print("Bader charges:")
print("Index    Symbol    Charge")
for i, charge in enumerate(charges):
    print(f"{i:5d}    {atoms.get_chemical_symbols()[i]:5s}    {charge:5.3f}")


Bader charges:
Index    Symbol    Charge
    0    O        7.148
    1    H        0.426
    2    H        0.426
